In [218]:
import os
import math
import numpy as np
import pandas as pd
import datetime
import time
import random
from datetime import date
import pandas_ta as ta
from ta.volatility import BollingerBands
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from yahoo_fin import stock_info as si
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
import fix_yahoo_finance as yf

In [219]:
def load_historic_data(symbol):
    today = datetime.date.today()
    today_str = today.strftime("%Y-%m-%d")
    #  Get last year's data
    start_date = today - (600 * US_BUSINESS_DAY)
    start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
    # Download data from Yahoo Finance
    try:
        df = si.get_data(symbol, start_date=start_date_str, end_date=today_str, index_as_date=False)
        return df
    except:
        print('Error loading stock data for ' + symbol)
        return None

In [220]:
def calculate_smas(df):
    df['SMA50'] = ta.sma(df["close"], length=50)
    df['SMA200'] = ta.sma(df["close"], length=200)

    return df

In [221]:
def determine_primary_market(df):
    df['BEAR'] = np.where(df['SMA50'] < df["SMA200"], df['close'], np.NaN)
    df['BULL'] = np.where(df['SMA50'] > df["SMA200"], df['close'], np.NaN) 

    return df

In [222]:
def plot_graph(df):
    fig = make_subplots(rows=1, cols=1)

    #  Plot close price
    fig.add_trace(go.Line(x = df.index, y = df['close'], line=dict(color="blue", width=1), name="Close"), row = 1, col = 1)
    
    #  Plot SMAs
    fig.add_trace(go.Line(x = df.index, y = df['SMA50'], line=dict(color="green", width=1), name="SMA50"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = df['SMA200'], line=dict(color="red", width=1), name="SMA200"), row = 1, col = 1)
    
    
    # Primary Markets
    fig.add_trace(go.Line(x = df.index, y = df['BULL'], line=dict(color="green", width=1), name="Bull"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = df['BEAR'], line=dict(color="red", width=1), name="Bear"), row = 1, col = 1)
    
    fig.update_layout(
        title={'text':'Primary Markets', 'x':0.5},
        autosize=False,
        width=800,height=400)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()
    

In [223]:
df = load_historic_data('NDX')
df = calculate_smas(df)
df = determine_primary_market(df)
plot_graph(df)

C:\Users\dutch\anaconda3\envs\tradesystem1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [224]:
#  Index Calculation
def load_crypto_data(symbols):
    today = datetime.date.today()
    today_str = today.strftime("%Y-%m-%d")
    #  Get last year's data
    start_date = today - (600 * US_BUSINESS_DAY)
    start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
    # Download data from Yahoo Finance
    try:
        df = si.get_data(symbols, start_date=start_date_str, end_date=today_str, index_as_date=False)
        return df
    except:
        print('Error loading stock data for ' + symbols)
        return None

In [225]:
symbols = ['XLM-USD','DOGE-USD','STX-USD']

start = datetime.datetime(2021,5,1)
end = datetime.datetime(2022,4,29)
crypto_df = yf.download(symbols, start=start, end=end)
#print(crypto_df.head())
#
#
#  Convert columns from tuple to strings
crypto_df.columns = ['{}_{}'.format(x[0], x[1]) for x in crypto_df.columns]
#for (columnName, columnData) in crypto_df.iteritems():
#    print('Column Name : ', columnName)
column_names = ['Close_STX-USD', 'Close_DOGE-USD', 'Close_XLM-USD']  
crypto_df['CRYPTO_INDEX']= crypto_df[column_names].sum(axis=1) / len(symbols)
print(crypto_df['CRYPTO_INDEX'])

[*********************100%***********************]  3 of 3 completed
Date
2021-05-01    1.025609
2021-05-02    1.002909
2021-05-03    1.020647
2021-05-04    1.018919
2021-05-05    1.130237
                ...   
2022-04-24    0.480810
2022-04-25    0.487341
2022-04-26    0.450578
2022-04-27    0.458952
2022-04-28    0.457430
Name: CRYPTO_INDEX, Length: 363, dtype: float64


In [226]:
def plot_crypto_index(df):
    fig = make_subplots(rows=1, cols=1)

    #  Plot close price
    fig.add_trace(go.Line(x = df.index, y = df['CRYPTO_INDEX'], line=dict(color="blue", width=2), name="Crypto Index"), row = 1, col = 1)
    
    #  Plot Coins
    fig.add_trace(go.Line(x = df.index, y = df['Close_STX-USD'], line=dict(color="red", width=1), name="STX-USD"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = df['Close_DOGE-USD'], line=dict(color="yellow", width=1), name="DOGE-USD"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = df['Close_XLM-USD'], line=dict(color="purple", width=1), name="XLM-USD"), row = 1, col = 1)
 
    fig.update_layout(
        title={'text':'Primary Markets', 'x':0.5},
        autosize=False,
        width=800,height=400)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()

In [227]:
plot_crypto_index(crypto_df)

C:\Users\dutch\anaconda3\envs\tradesystem1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


